In [2]:
from py_ecc.bn128 import add, multiply, neg, curve_order, G1, Z1

# BASIC OPERATIONS

In [4]:
# Generator point
print("GENERATOR POINT:", G1, "\n\n")

# Add points
print("POINT SUM:", add(G1, G1), "\n\n")

# Multiply point
print("POINT MULTIPLICATION:", multiply(G1, 2), "\n\n")

GENERATOR POINT: (1, 2) 


POINT SUM: (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764) 


POINT MULTIPLICATION: (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764) 




# POINT AT INFINITY AND WRAP AROUND

In [25]:
# Adding a point and its inverse gets you the point at infinity
assert add(G1, neg(G1)) is None

# If you add G1 to itself curve_order times, you get G1
assert add(G1, multiply(G1, curve_order)) == G1

(1368015179489954701390400359078579693043519447331113978918064868415326638035, 11970132820537103637166003141937572314130795164147247315533067598634108082819)


# Exercise 1
### Get inverse without using neg function

In [6]:
field_element = 5
GN = multiply(G1, field_element)

# METHOD 1:
y_val = -GN[1]
GN_inv = (GN[0], y_val)
assert neg(GN) == GN_inv

# METHOD 2:
inverse_field_element = curve_order - field_element
GN_inv = multiply(G1, inverse_field_element)
assert neg(GN) == GN_inv

# Exercise 2
### ZK Addition: I know two numbers that sum to 16.

In [ ]:
"""PROVER:""" 
A = multiply(G1, 5)
B = multiply(G1, 11)
proof = (A, B, 16)

"""VERIFIER:"""
v_point = multiply(G1, 16)
assert add(A, B) == v_point

# Exercise 3
### ZK Multiplication by Constant: "I know that 5 times a hidden number, H, is 55."

In [ ]:

# Prover will:
hidden_number = 11
H = multiply(G1, hidden_number)
proof = (5, H, 55)

"""VERIFIER:"""
P = multiply(G1, 55)
assert multiply(H, 5) == P

# Exercise 4
### ZK Equation: "I know x and y such that 5x + 2y = 16"

In [ ]:
"""PROVER:""" 
x = 2
y = 3

X = multiply(G1, x)
Y = multiply(G1, y)
proof = (5, X, 2, Y, 16)

"""VERIFIER:"""

construct = add(multiply(X, 5), multiply(Y, 2))
assert construct == multiply(G1, 16)

# Exercise 5
### ZK Rational Number Math: how to do the following on Elliptic Curves?

$$  5/2 + 1/2 = 3 $$

$$ 5 * (1/2) + (1) * (1/2) = 3 $$

In [5]:
inv_field_elem = pow(2, -1, curve_order)

inv2_point = multiply(G1, inv_field_elem)

checkpoint = multiply(G1, 3)

assert add(multiply(inv2_point, 5), inv2_point) == checkpoint


# HW3 Exercise 1

Rationale:

I can't deivide the numerator by the denominator due to the integer division:

$$ A + B = G1 * (num/den) $$

I can, however, rearrange the equation:

$$ (A + B) = G1 * num * den^{-1} \pmod {curve\_order} $$ 

### Example Problem:

$$ 5/2 + 3/2 = 8/2 = 4 $$

In [5]:
CURVE_ORDER = 21888242871839275222246405745257275088548364400416034343698204186575808495617

"""PROVER:""" 
num = 4
den = 1

# 1. Re-represent problem into: 5 * inv(2) + 3 * inv(2) = 4
r_den_a = 2 # rational denominator
r_den_b = 2 # rational denominator

# 2. Find rational denominator's modular inverse
r_den_a_inverse = pow(r_den_a, CURVE_ORDER - 2, CURVE_ORDER)
r_den_b_inverse = pow(r_den_b, CURVE_ORDER - 2, CURVE_ORDER)

# 3. Construct A and B using inverses and scalars
inv_a_point = multiply(G1, r_den_a_inverse)
inv_b_point = multiply(G1, r_den_b_inverse)
A = multiply(inv_a_point, 5)
B = multiply(inv_b_point, 3)
print("A:", A)
print("B:", B)

# 4. Construct proof
proof = (A, B, num, den)

"""VERIFIER:"""
A = proof[0]
B = proof[1]
num = proof[2]
den = proof[3]

# 5. Construct C: num/den can be represented as num * inv(den)
inv_den = pow(den, CURVE_ORDER - 2, CURVE_ORDER)
inv_den_point = multiply(G1, inv_den)
C = multiply(inv_den_point, num)

# 6. Verify that A + B == C
assert add(A, B) == C

A: (19032580475487806326057692075690361508625732333378927815997075982203596960703, 8731032473663224878408972807329716494736032188868213550913069617079965841757)
B: (20218473672104602182365268103669367305741558787574619646654823060903696598123, 1222619430156132345159311921052235294370050724750479676095071048268376132670)


# HW3 Exercise 2 

In [17]:
# construct m
n = 4
m = []
for i in range(n * n):
    m.append(i+1)

print("m:", m, "\n\n")


# construct test points for s
s = []
i = 2
for _ in range(n):
    s.append(multiply(G1, i))
    i *= 2

# print for copy to remix
s_list = []
for point in s:
    s_list.append(list(point))

print("s_list:", s_list)


# construct test o
o = []
for row in range(n):
    POINT = Z1
    for col in range(n):
        elem = m[row * n + col]
        POINT = add(POINT, multiply(s[col], elem)) 
    o.append(POINT)

# print for copy to remix
o_list = []
for point in o:
    o_list.append(list(point))

print("o_list:", o_list)




m: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16] 


s_list: [[1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764], [3010198690406615200373504922352659861758983907867017329644089018310584441462, 4027184618003122424972590350825261965929648733675738730716654005365300998076], [3932705576657793550893430333273221375907985235130430286685735064194643946083, 18813763293032256545937756946359266117037834559191913266454084342712532869153], [10835225521862395592687560951453385602895512958032257955899877380493200080708, 2623520004791921319615054428233368525468155544765295675952919303096698181037]]
o_list: [[36435015165795949864190500573076466772343558715483085869477662269995883502, 12578652896417066812813214699834299212164767626159614960854168133330558784127], [5208915768490282719853875436926270237244370626538455259071168950361949252324, 19217052291869819708375403416173486519711020651119